Used to test different deltas on different maps and models

Import Dependancies 

In [2]:
from vizdoom import * #Import all of vizdoom
import numpy as np #Numpy for identity matrix
import time #To make the program sleep (wait), so we can actually see what's happening
from stable_baselines3.common import env_checker #Import the env_checker class from stable_baselines3 to check the environment
from stable_baselines3 import PPO #Import the PPO class for training
from stable_baselines3.common.evaluation import evaluate_policy #Import the evaluate_policy function to evaluate the model
import os #To save the model to the correct path
from vizdoom_with_ai_gym_env_test import TrainAndLogCallback #Import the environment class and TrainAndLogCallback 
from pathfinder import doomfinder, create_new_checkpoint_directory
from gymnasium import Env #Import OpenAI Gym's Env class
from gymnasium.spaces import Discrete, Box #Import OpenAI Gym's Discrete and Box spaces
import cv2 #OpenCV for image processing, used for modifying the DOOM environment to make it run faster 

Testing basic.cfg (using deltas)

In [36]:
class VizDoomGym_Simple(Env): 
    def __init__(self, config_path, render=False): #Constructor
        
        #Configs this is used for: basic.cfg, defend_the_center.cfg

        super(VizDoomGym_Simple, self).__init__() #Inherit from Env class

        #Args: 
            #config_path (str): The path to the configuration file
            #render (bool): Whether to render the environment or not, false by default

        #Setup game
        self.game = vizdoom.DoomGame() #Create a DoomGame object
        self.game.load_config(config_path) #Load the configuration file from file path, ex: doomfinder("basic.cfg")

        #Set window visibility
        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)

        self.game.init() #Start the game

        #Setup action and observation space
        self.observation_space = Box(low=0, high=255, shape=(100, 160, 1), dtype=np.uint8) #Observation space, 100x160x1 image
        self.action_space = Discrete(3) #Action space, 3 actions

        #Game variables
        self.ammo = self.game.get_state().game_variables[0]  #Get the ammo count, initialize to the current ammo
        self.health = 100 #Initialize health to 100 (assuming we start at full health)
        print("Initialized VizDoomGym_Simple environment.")
        print(self.ammo, self.health)

    def step(self, action, limit = 1000): #Take a step in the environment 
        #Args:
            #action (int): The action to take
            #limit (int): Unimplemented "limit" for the episode, most likely will be a time limit
        #Returns:
            #observation (np.array): The screen buffer of the environment
            #reward (float): The reward for the action taken
            #terminated (bool) Whether the episode is finished or not (by reaching the goal)
            #truncated (bool): Whether the episode has reached some terminal state without reaching the goal (ie: running out of time)
            #info (dict): Additional information about the environment

        #Specify actions and take a step
        actions = np.identity(3) #Create an identity matrix with 3 rows (3 actions), MOVE_LEFT, MOVE_RIGHT, ATTACK, these are the actions we can take in the environment
        movement_reward = self.game.make_action(actions[action], 4) #Reward for taking a random action, second parameter is frame skip (skip 4 frames before taking the next action), the reason we do this is because it saves us time while being easy to see what is happening 
        reward = movement_reward #Initialize reward to movement reward
        truncated = False #Not implemented yet, so set to False. The idea is that if step passes some sort of limit, like a time limit, then the episode is truncated.
        info = {} #Initialize info to an empty dictionary
        Basic = False

        if self.game.get_state(): #If the game is not finished
            observation = self.game.get_state().screen_buffer #Get the screen buffer
            observation = self.greyscale(observation) #Convert the image to greyscale

            #Get game variables
            game_variables = self.game.get_state().game_variables
            if len(game_variables) == 2:
                ammo, health = game_variables
            else:
                ammo = game_variables[0]
                health = 100 #Assume health is 100 if it's not provided (effectively ignoring it)
                Basic = True
            
            #Calculate reward deltas
            if(Basic == False): #If its the basic config we just ignore all deltas entirely, I know this is janky but its just a testing enviorment so whatever
                ammo_delta = ammo - self.ammo #Current ammo - old ammo = ammo used
                ammo = self.ammo 
                health_delta = health - self.health  #Current health - old health = damage taken
                health = self.health
            
                reward = movement_reward + ammo_delta*0.1 + health_delta*0.2 #Calculate the reward
                print(f"Health delta: {health_delta}, Ammo delta: {ammo_delta}, Movement reward: {movement_reward}, Reward: {reward} \n")
            info = {"ammo": ammo}
        else:
            observation = np.zeros(self.observation_space.shape) #Return a blank screen
            print(f"Movement reward: {movement_reward}, Reward: {reward} \n")

        terminated = self.game.is_episode_finished() #Check if the episode is finished

        return observation, reward, terminated, truncated, info



    def render(self, render_in_greyscale=False): #Render the environment for a frame
        #Args:
            #render_in_greyscale (bool): Whether to render the environment in greyscale or not
        
        if self.game.get_state() and render_in_greyscale:  #Only render if there's a valid game state
            observation = self.game.get_state().screen_buffer
            greyscale_obs = self.greyscale(observation)  #Convert to greyscale
            #Render using OpenCV to visualize
            cv2.imshow("VizDoom Environment", greyscale_obs.squeeze())  #Remove extra dimension and display
            cv2.waitKey(1)  #Wait 1ms between frames to allow for rendering
        elif self.game.get_state():  #Only render if there's a valid game state
            observation = self.game.get_state().screen_buffer
            #Render using OpenCV to visualize
            cv2.imshow("VizDoom Environment", observation.squeeze())  #Remove extra dimension and display
            cv2.waitKey(1)  #Wait 1ms between frames to allow for rendering
        else:
            print("No game state to render.")

            
    def reset(self, seed=None): #Reset the environment when we start a new game
        #Args:
            #seed (int): The seed for the random number generator
        #Returns:
            #(observation, info) (tuple)
                #observation (np.array): The screen buffer of the environment
                #info (dict): Additional information about the environment
            
        super().reset(seed=seed) #Implement seeding
        
        self.game.new_episode() #Start a new episode
        state = self.game.get_state().screen_buffer #Get the screen buffer
        observation = self.greyscale(state) #Convert the image to greyscale
        
        #Gather any additional environment-specific info (like ammo, etc.)
        if self.game.get_state():
            ammo = self.game.get_state().game_variables[0]  #Get the ammo count
            info = {"ammo": ammo}
        else:
            info = {} #No gamestate means no info can be gathered
        
        return (observation, info) #Tuple of observation and info

    def greyscale(self, observation=None): #Convert the enivornment to greyscale and resize it
        #Args:
            #observation (np.array): The image of the environment (the current game frame)
        #Returns:
            #grey_return (np.array): The resized greyscale image of the environment
        
        if observation is None and self.game.get_state(): #If no observation is passed
            observation = self.game.get_state().screen_buffer #Get the screen buffer 

        grey = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY) #Convert the image to greyscale
        resize = cv2.resize(grey, (160, 100), interpolation=cv2.INTER_CUBIC) #Resize the image to 160x100
        state = np.reshape(resize, (100, 160, 1)) #Reshape the image to 100x160x1
        
        return state
    
    def get_state(self): 
        #Returns:
            #state (np.array): The current state of the environment
        return self.game.get_state()

    def close(self): #Close the environment
        self.game.close()
        
env = VizDoomGym_Simple(config_path=doomfinder('basic.cfg'), render=True) #Reload env with new map

model = PPO.load('./Training/checkpoints/best_model_PPO_test_basic_2/best_model_500000.zip') #Load the model (hardcoded to load a specific model but adjust as needed)

for episode in range(5):
    obs, _ = env.reset()  #Reset the environment and get only the observation
    done = False  #Set done to false
    total_reward = 0  #Set total reward to 0
    while not done:  #While the game isn't done
        action, _ = model.predict(obs)  #Get the action
        obs, reward, done, truncated, info = env.step(action)  #Take the action
        total_reward += reward  #Add the reward to the total reward
        time.sleep(0.05)  #Sleep for 0.05 seconds
    #print('Episode: {}, Total Reward: {}'.format(episode, total_reward))  #Print the episode and total reward
    time.sleep(2)  #Sleep for 2 seconds


Initialized VizDoomGym_Simple environment.
50.0 100


UnboundLocalError: cannot access local variable 'health_delta' where it is not associated with a value

Testing defend_the_center.cfg

In [28]:
class VizDoomGym_Simple(Env): 
    def __init__(self, config_path, render=False): #Constructor
        
        #Configs this is used for: basic.cfg, defend_the_center.cfg

        super(VizDoomGym_Simple, self).__init__() #Inherit from Env class

        #Args: 
            #config_path (str): The path to the configuration file
            #render (bool): Whether to render the environment or not, false by default

        #Setup game
        self.game = vizdoom.DoomGame() #Create a DoomGame object
        self.game.load_config(config_path) #Load the configuration file from file path, ex: doomfinder("basic.cfg")

        #Set window visibility
        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)

        self.game.init() #Start the game

        #Setup action and observation space
        self.observation_space = Box(low=0, high=255, shape=(100, 160, 1), dtype=np.uint8) #Observation space, 100x160x1 image
        self.action_space = Discrete(3) #Action space, 3 actions

        #Game variables
        self.ammo = self.game.get_state().game_variables[0]  #Get the ammo count, initialize to the current ammo
        self.health = 100 #Initialize health to 100 (assuming we start at full health)
        self.hitcount = 0 #Initialize hitcount to 0
        print("Initialized VizDoomGym_Simple environment.")
        print(self.ammo, self.health, self.hitcount)


    def step(self, action, limit = 1000): #Take a step in the environment 
        #Args:
            #action (int): The action to take
            #limit (int): Unimplemented "limit" for the episode, most likely will be a time limit
        #Returns:
            #observation (np.array): The screen buffer of the environment
            #reward (float): The reward for the action taken
            #terminated (bool) Whether the episode is finished or not (by reaching the goal)
            #truncated (bool): Whether the episode has reached some terminal state without reaching the goal (ie: running out of time)
            #info (dict): Additional information about the environment

        #Specify actions and take a step
        actions = np.identity(3) #Create an identity matrix with 3 rows (3 actions), MOVE_LEFT, MOVE_RIGHT, ATTACK, these are the actions we can take in the environment
        movement_reward = self.game.make_action(actions[action], 4) #Reward for taking a random action, second parameter is frame skip (skip 4 frames before taking the next action), the reason we do this is because it saves us time while being easy to see what is happening 
        reward = 0 #Initialize reward to 0
        truncated = False #Not implemented yet, so set to False. The idea is that if step passes some sort of limit, like a time limit, then the episode is truncated.
        info = {} #Initialize info to an empty dictionary

        if self.game.get_state(): #If the game is not finished
            observation = self.game.get_state().screen_buffer #Get the screen buffer
            observation = self.greyscale(observation) #Convert the image to greyscale

            #Get game variables
            game_variables = self.game.get_state().game_variables
            if len(game_variables) == 2:
                ammo, health = game_variables
            if len(game_variables) == 3:
                ammo, health, hitcount = game_variables
                hitcount_delta = hitcount - self.hitcount
                self.hitcount = hitcount
            else:
                ammo = game_variables[0]
                health = 100 #Assume health is 100 if it's not provided (effectively ignoring it)
                hitcount_delta = 0 #Assume hitcount is 0 if it's not provided (effectively ignoring it)
            
            #Calculate reward deltas
            ammo_delta = ammo - self.ammo #Current ammo - old ammo = ammo used
            ammo = self.ammo 
            health_delta = health - self.health  #Current health - old health = damage taken
            health = self.health
            hitcount_delta = hitcount - self.hitcount
            hitcount = self.hitcount


            reward = movement_reward + ammo_delta*0.1 + health_delta*0.2 + hitcount_delta #Calculate the reward
            info = {"ammo": ammo}
        else:
            observation = np.zeros(self.observation_space.shape) #Return a blank screen
            health_delta = 0
            ammo_delta = 0
            hitcount_delta = 0

        terminated = self.game.is_episode_finished() #Check if the episode is finished

        
        print(f"Health delta: {health_delta}, Ammo delta: {ammo_delta}, Hitcount delta: {hitcount_delta}, Movement Reward {movement_reward}, Reward: {reward} \n")

        return observation, reward, terminated, truncated, info

    def render(self, render_in_greyscale=False): #Render the environment for a frame
        #Args:
            #render_in_greyscale (bool): Whether to render the environment in greyscale or not
        
        if self.game.get_state() and render_in_greyscale:  #Only render if there's a valid game state
            observation = self.game.get_state().screen_buffer
            greyscale_obs = self.greyscale(observation)  #Convert to greyscale
            #Render using OpenCV to visualize
            cv2.imshow("VizDoom Environment", greyscale_obs.squeeze())  #Remove extra dimension and display
            cv2.waitKey(1)  #Wait 1ms between frames to allow for rendering
        elif self.game.get_state():  #Only render if there's a valid game state
            observation = self.game.get_state().screen_buffer
            #Render using OpenCV to visualize
            cv2.imshow("VizDoom Environment", observation.squeeze())  #Remove extra dimension and display
            cv2.waitKey(1)  #Wait 1ms between frames to allow for rendering
        else:
            print("No game state to render.")

            
    def reset(self, seed=None): #Reset the environment when we start a new game
        #Args:
            #seed (int): The seed for the random number generator
        #Returns:
            #(observation, info) (tuple)
                #observation (np.array): The screen buffer of the environment
                #info (dict): Additional information about the environment
            
        super().reset(seed=seed) #Implement seeding
        
        self.game.new_episode() #Start a new episode
        state = self.game.get_state().screen_buffer #Get the screen buffer
        observation = self.greyscale(state) #Convert the image to greyscale
        
        #Gather any additional environment-specific info (like ammo, etc.)
        if self.game.get_state():
            ammo = self.game.get_state().game_variables[0]  #Get the ammo count
            info = {"ammo": ammo}
        else:
            info = {} #No gamestate means no info can be gathered
        
        return (observation, info) #Tuple of observation and info

    def greyscale(self, observation=None): #Convert the enivornment to greyscale and resize it
        #Args:
            #observation (np.array): The image of the environment (the current game frame)
        #Returns:
            #grey_return (np.array): The resized greyscale image of the environment
        
        if observation is None and self.game.get_state(): #If no observation is passed
            observation = self.game.get_state().screen_buffer #Get the screen buffer 

        grey = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY) #Convert the image to greyscale
        resize = cv2.resize(grey, (160, 100), interpolation=cv2.INTER_CUBIC) #Resize the image to 160x100
        state = np.reshape(resize, (100, 160, 1)) #Reshape the image to 100x160x1
        
        return state
    
    def get_state(self): 
        #Returns:
            #state (np.array): The current state of the environment
        return self.game.get_state()

    def close(self): #Close the environment
        self.game.close()
        
env = VizDoomGym_Simple(config_path=doomfinder('defend_the_center_hitcount_test.cfg'), render=True) #Reload env with new map

model = PPO.load('./Training/checkpoints/best_model_PPO_test_defend_the_center_1/best_model_100000.zip') #Load the model (hardcoded to load a specific model but adjust as needed)

for episode in range(5):
    obs, _ = env.reset()  #Reset the environment and get only the observation
    done = False  #Set done to false
    total_reward = 0  #Set total reward to 0
    while not done:  #While the game isn't done
        action, _ = model.predict(obs)  #Get the action
        obs, reward, done, truncated, info = env.step(action)  #Take the action
        total_reward += reward  #Add the reward to the total reward
        time.sleep(0.05)  #Sleep for 0.05 seconds
    #print('Episode: {}, Total Reward: {}'.format(episode, total_reward))  # Print the episode and total reward
    time.sleep(2)  #Sleep for 2 seconds


Initialized VizDoomGym_Simple environment.
26.0 100 0
Health delta: 0.0, Ammo delta: 0.0, Hitcount delta: 0.0, Movement Reward 0.0, Reward: 0.0 

Health delta: 0.0, Ammo delta: 0.0, Hitcount delta: 0.0, Movement Reward 0.0, Reward: 0.0 

Health delta: 0.0, Ammo delta: -1.0, Hitcount delta: 0.0, Movement Reward 1.0, Reward: 0.9 

Health delta: 0.0, Ammo delta: -1.0, Hitcount delta: 0.0, Movement Reward 0.0, Reward: -0.1 

Health delta: 0.0, Ammo delta: -1.0, Hitcount delta: 0.0, Movement Reward 0.0, Reward: -0.1 

Health delta: 0.0, Ammo delta: -1.0, Hitcount delta: 0.0, Movement Reward 0.0, Reward: -0.1 

Health delta: 0.0, Ammo delta: -1.0, Hitcount delta: 0.0, Movement Reward 0.0, Reward: -0.1 

Health delta: 0.0, Ammo delta: -1.0, Hitcount delta: 0.0, Movement Reward 0.0, Reward: -0.1 

Health delta: 0.0, Ammo delta: -1.0, Hitcount delta: 0.0, Movement Reward 0.0, Reward: -0.1 

Health delta: 0.0, Ammo delta: -1.0, Hitcount delta: 0.0, Movement Reward 0.0, Reward: -0.1 

Health delt

It doesnt seem to consider the hitcount as a game variable, so it's not being used in the reward calculation, I need to figure out how to make it consider the hitcount as a game variable later, or at least why it isnt
